In [1]:
import os
import tensorflow as tf


In [2]:
os.chdir("../")

In [3]:
model= tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    params_image_size:list
    params_batch_size:int
    

In [21]:
from firstClassifier.constants import *
from firstClassifier.utils import read_yaml, create_directories
class ConfigurationManager:

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
             ):
        """_summary_: Creates artifact directories taken from config.yaml file
        """
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        ##Create directories
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)->EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/PetImages",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [32]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path
from firstClassifier.utils import save_json
#from firstClassifier.entity import EvaluationConfig


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config= config

    
    def _valid_generator(self):

        ##Augmentation
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split= 0.30

        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

        
    def evaluation(self):
        model= self.load_model(self.config.path_of_model)
        self._valid_generator()
        ##score returns loss and accuracy
        self.score= model.evaluate(
            self.valid_generator
        )

    def save_score(self):
        scores= {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data= scores)



In [33]:
try:
    config= ConfigurationManager()
    validation_config= config.get_validation_config()
    evaluate= Evaluation(validation_config)
    evaluate.evaluation()
    evaluate.save_score()

except Exception as e:
    raise e

[2023-04-13 19:25:24,881: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2023-04-13 19:25:24,884: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-13 19:25:24,885: INFO: common]: created directory at: artifacts
Found 60 images belonging to 2 classes.
4/4 [==============================] - 12s 3s/step - loss: 17.8638 - accuracy: 0.5000
[2023-04-13 19:25:36,789: INFO: common]: json file saved at: scores.json
